# Redis Chat Message History

This notebook demonstrates how to use the `RedisChatMessageHistory` class from the `langchain-redis` package to efficiently store, fetch, and manage chat message history in [Redis](https://redis.io).

## Setup

First, we need to install the required dependencies and ensure we have a Redis instance running.

In [ ]:
%pip install -qU langchain-redis langchain-openai

Make sure you have a [Redis](https://redis.io) server running. You can start one using Docker with the following command:

```
docker run -d -p 6379:6379 redis/redis-stack-server:latest
```

Or install and run Redis locally according to the instructions for your operating system.

In [1]:
# ruff: noqa: T201
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


## Importing Required Libraries

In [2]:
import logging

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

from langchain_redis import RedisChatMessageHistory

logging.getLogger('redisvl').setLevel(logging.WARNING)

## Basic Usage of RedisChatMessageHistory

In [3]:
# Initialize RedisChatMessageHistory
history = RedisChatMessageHistory(session_id="user_123", redis_url=REDIS_URL)

# Add messages to the history
history.add_user_message("Hello, AI assistant!")
history.add_ai_message("Hello! How can I assist you today?")
history.add_user_message("How are you today?")
history.add_ai_message("I am doing well today. How can I help?")

# Retrieve messages
print("Chat History:")
for message in history.messages:
    print(f"{type(message).__name__}: {message.content}")

Chat History:
HumanMessage: Hello, AI assistant!
AIMessage: Hello! How can I assist you today?
HumanMessage: How are you today?
AIMessage: I am doing well today. How can I help?


In [4]:
# Take a look at the chat history index schema
history.index.schema

IndexSchema(index=IndexInfo(name='idx:chat_history', prefix='chat:', key_separator=':', storage_type=<StorageType.JSON: 'json'>), fields={'session_id': TagField(name='session_id', type=<FieldTypes.TAG: 'tag'>, path='$.session_id', attrs=TagFieldAttributes(sortable=False, separator=',', case_sensitive=False, withsuffixtrie=False)), 'content': TextField(name='content', type=<FieldTypes.TEXT: 'text'>, path='$.data.content', attrs=TextFieldAttributes(sortable=False, weight=1, no_stem=False, withsuffixtrie=False, phonetic_matcher=None)), 'type': TagField(name='type', type=<FieldTypes.TAG: 'tag'>, path='$.type', attrs=TagFieldAttributes(sortable=False, separator=',', case_sensitive=False, withsuffixtrie=False)), 'timestamp': NumericField(name='timestamp', type=<FieldTypes.NUMERIC: 'numeric'>, path='$.timestamp', attrs=NumericFieldAttributes(sortable=False))}, version='0.1.0')

In [5]:
history.clear()

## Using RedisChatMessageHistory with Language Models

### Set OpenAI API key

In [6]:
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("Please enter your OpenAI API key: ")

In [7]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# Create the conversational chain
chain = prompt | llm


# Function to get or create a RedisChatMessageHistory instance
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, redis_url=REDIS_URL)


# Create a runnable with message history
chain_with_history = RunnableWithMessageHistory(
    chain, get_redis_history, input_messages_key="input", history_messages_key="history"
)

In [12]:
# Use the chain in a conversation
response1 = chain_with_history.invoke(
    {"input": "Hi, my name is Tyler."},
    config={"configurable": {"session_id": "tyler_123"}},
)
print("AI Response 1:", response1.content)

09:01:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
AI Response 1: Hi Tyler! How can I assist you today?


In [13]:
response2 = chain_with_history.invoke(
    {"input": "What's my name?"}, config={"configurable": {"session_id": "tyler_123"}}
)
print("AI Response 2:", response2.content)

09:01:23 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
AI Response 2: Your name is Tyler. How can I help you today, Tyler?


In [14]:
# Check the messages
history = RedisChatMessageHistory("tyler_123", redis_url=REDIS_URL)
history.messages

[HumanMessage(content='Hi, my name is Tyler.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Tyler! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={}),
 HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Tyler. How can I help you today, Tyler?', additional_kwargs={'refusal': None}, response_metadata={})]

In [15]:
# Clear history for the session
history.clear()

## Advanced Features

### Custom Redis Configuration

In [16]:
# Initialize with custom Redis configuration
custom_history = RedisChatMessageHistory(
    session_id="user_456",
    redis_url=REDIS_URL,
    key_prefix="my_chat:",
    ttl=3600,  # Set TTL to 1 hour
    index_name="chat_idx",
)

custom_history.add_user_message("This is a message with custom configuration.")
print("Custom History:", custom_history.messages)

Custom History: [HumanMessage(content='This is a message with custom configuration.', additional_kwargs={}, response_metadata={}), HumanMessage(content='This is a message with custom configuration.', additional_kwargs={}, response_metadata={})]


### Searching Messages

In [17]:
# Add more messages
history.add_user_message("Tell me about artificial intelligence.")
history.add_ai_message(
    "Artificial Intelligence (AI) is a branch of computer science..."
)

# Search for messages containing a specific term
search_results = history.search_messages("artificial intelligence")
print("Search Results:")
for result in search_results:
    print(f"{result['type']}: {result['content'][:50]}...")

Search Results:
human: Tell me about artificial intelligence....
ai: Artificial Intelligence (AI) is a branch of comput...


In [18]:
# Clear the chat history (only current session)
history.clear()

print("Messages after clearing:", history.messages)

Messages after clearing: []


In [19]:
# Clear AND delete the chat history index completely (all sessions affected)
history.delete()

## Conclusion

This notebook demonstrated the key features of `RedisChatMessageHistory` from the langchain-redis package. It showed how to initialize and use the chat history, integrate it with language models, and utilize advanced features like custom configurations and message searching. Redis provides a fast and scalable solution for managing chat history in AI applications.